In [2]:
import numpy as np
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20_20
output_notebook()

Loading BokehJS ...

# Cycle Time Machine
We just have turned on our debt cycle today, go ahead and add one character to see what happens!

In [5]:
np.random.seed(42)
def get_size_for_chars(chars):
    x1, x2, y1, y2 = 1, 50, 15, 5
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1

    size = int(m * chars + b)
    if size < 5:
        return 5
    return size

DTS = [0, 1, 5, 8, 11, 15, 17, 25, 28, 33, 34, 38, 40]
def create_char_history(name="SpongeBob", offset=0, nc=0, nc_point=17, char_id=1, eoc=-1):
    char = {
        "name": f"{name}",
        "interactions": [
            f"☎️ Hey {name}, look, we have some communication for you!",
            f"☎️ Hey {name}, it's us again, probably you didn't see yesterday msg",
            f"💌 Hey {name}, we're trying to contact you, please call us back or answer this email.",
            f"☎️ Hey {name}, it seems that we are not able to get in touch, please call us back",
            f"💌 Hey {name}, we need to talk",
            "💌 Harder msg 1",
            "💌 Harder msg 2",
            "💌 Worrying warning",
            "💌 A bit more worrying warning",
            "☎️ This is a serious message",
            "💌 One of the last opportunities",
            "💌 Yeah, seriously",
            "☎️ Last call",
        ],
        "dt": DTS,
        "nc_point": 0,
        "char_id": char_id,
    }
    data = pd.DataFrame(char)

    # classify means
    sms = data.interactions.str.contains("☎️")
    data["comm_mean"] = "email"
    data.loc[sms, "comm_mean"] = "sms"


    if nc:
        # Add the points where the non contactable history kicks in
        index = DTS.index(nc_point)
        nc_point_next = DTS[index + 1]
        for p, v in zip((nc_point, nc_point_next), (1, 2)):
            c = data["dt"] == p
            data.loc[c, "nc_point"] = v

        # Add some non contactable point
        c = data["dt"] > nc_point
        data.loc[c, "dt"] += nc
        data["dt"] += offset
        return data

    data["dt"] += offset
    return data[data.dt <= DTS[eoc]]

def create_cycle():
    spongebob = create_char_history()
    patrick_star = create_char_history("Patrick Star", offset=1, char_id=2)
    krabs = create_char_history("MR Krabs", offset=3, char_id=3, eoc=3)
    squidward = create_char_history("Squidward", offset=1, nc=10, char_id=4)
    cast = [spongebob, patrick_star, krabs, squidward, ]
    for c in range(5, 100):
        char = create_char_history(
            name=f"extra_tt_{c - 3}",
            offset=np.random.randint(1, 20),
            nc=np.random.poisson(lam=1) * 5,
            nc_point=DTS[np.random.randint(6, 8)],
            char_id=c,
            eoc=np.random.randint(8, len(DTS))
        )
        cast.append(char)
    return pd.concat(cast)

k0 = create_cycle()

def plot(characters=1, days=0):
    k1 = k0[(k0["dt"] <= days) & (k0["char_id"] <= characters)]

    k3 = k1.groupby(["name", "char_id"]).dt.max().reset_index()
    k3["dt_end"] = days

    s0 = ColumnDataSource(k1)

    if characters > 20:
        factors = (1, characters + 1)
        y = "char_id"
        y_segment = k3.char_id
        TOOLTIPS = None
    else:
        factors = k1.name.unique()
        y = "name"
        y_segment = k3.name
        TOOLTIPS = """<span style="font-size: 16px;">Day Since Turned on DC: @dt<br>@interactions</span>"""
    cmap = "grey"


    if characters <= 20:
        cmap = factor_cmap('name', palette=Category20_20, factors=factors)

    p = figure(
        width=1200, height=600, y_range=factors, title="Cycle Interactive Map", tooltips=TOOLTIPS
    )

    # Non contactable segments
    x0 = k0.loc[k0.nc_point == 1, "dt"]
    x1 = k0.loc[k0.nc_point == 2, "dt"]
    y0 = k0.loc[k0.nc_point == 1, y]
    y1 = k0.loc[k0.nc_point == 2, y]
    p.segment(x0=x0, x1=x1, y0=y0, y1=y1, line_width=3)

    # eoc segments
    p.segment(
        x0=k3.dt, x1=k3.dt_end, y0=y_segment, y1=y_segment, line_width=1, line_dash="dashed",
    )

    p.circle(
        x='dt', y=y, size=get_size_for_chars(characters), fill_color=cmap, source=s0)

    ticker = np.arange(days+1)
    if days > 40:
        ticker = np.arange(days, step=5)
    p.xaxis.ticker = ticker
    p.yaxis.axis_label = "Characters"
    p.xaxis.axis_label = "Days Since We Turned on the Cycle"
    p.yaxis.axis_label_text_font_size = "16px"
    p.title.text_font_size = "20px"
    p.xaxis.axis_label_text_font_size = "16px"
    p.yaxis.major_label_text_font_size = "16px"

    show(p)

characters = widgets.BoundedIntText(value=0, min=0, max=100, step=1)
days = widgets.BoundedIntText(value=0, min=0, max=200, step=5)
_ = interact(plot, characters=characters, days=days)
# plot(characters=10, days=40)

interactive(children=(BoundedIntText(value=0, description='characters'), BoundedIntText(value=0, description='…